In [1]:
import math
import random
import string
import sys
import time
import numpy as np
import torch

import matplotlib.pyplot as plt
import src
from envs import build_env
#from src.evaluator import Evaluator
from src.slurm import init_signal_handler, init_distributed_mode
from t2.realtime_trainer import RealtimeTrainer
from src.utils import initialize_exp, str_diff
from t2.utils import get_parser, join_sai, to_sai_str

from t2.transformer import build_transformer
argv = [
        '--exp_name', 'first_train', 
        '--tasks', 'add_dataset', 
        '--n_enc_layers', '16', 
        '--n_heads', '4', 
        '--sinusoidal_embeddings', 'true', 
        '--num_workers', '4', 
        '--eval_onl', '0', 
        '--save_periodic', '0', 
        '--epoch_size', '10000', 
        '--batch_size', '32', 
        '--dropout', '0.1', 
        '--attention_dropout', '0.1', 
        '--emb_dim', '64', 
        '--bottleneck_dim', '64', 
        '--nn_output', '1', 
        '--input_seq_length', '47', 
        '--share_inout_emb', 'false'
]

parser = get_parser()

params = parser.parse_args(argv)
params.my_device='cuda'

print(params)

bs = params.batch_size
#dim = params.emb_dim

init_distributed_mode(params)
logger = initialize_exp(params)

# CPU / CUDA
if params.cpu:
    assert not params.multi_gpu
else:
    assert torch.cuda.is_available()
    
src.utils.CUDA = not params.cpu

env = build_env(params)

modules = build_transformer(env, params)
trainer = RealtimeTrainer(modules, env, params)


INFO - 01/08/23 23:41:10 - 0:00:00 - ============ Initialized logger ============
INFO - 01/08/23 23:41:10 - 0:00:00 - accumulate_gradients: 1
                                     amp: -1
                                     attention_dropout: 0.1
                                     batch_size: 32
                                     beam_early_stopping: True
                                     beam_eval: False
                                     beam_length_penalty: 1
                                     beam_size: 1
                                     bottleneck_dim: 64
                                     clip_grad_norm: 5
                                     command: python F:\Programs\anaconda3\envs\ai\lib\site-packages\ipykernel_launcher.py '-f' 'C:\Users\Dmytro\AppData\Roaming\jupyter\runtime\kernel-657dd551-23fc-4593-8171-0748d559e9dc.json' --exp_id "dmytro_job"
                                     cpu: False
                                     debug: False
               

SAIEnvironment created
Namespace(dump_path='./dumped/', exp_name='first_train', save_periodic=0, exp_id='dmytro_job', fp16=False, amp=-1, emb_dim=64, bottleneck_dim=64, nn_output=1, input_seq_length=47, n_enc_layers=16, n_dec_layers=4, n_heads=4, dropout=0.1, attention_dropout=0.1, share_inout_emb=False, sinusoidal_embeddings=True, env_base_seed=0, max_len=512, batch_size=32, optimizer='adam,lr=0.0001', clip_grad_norm=5, epoch_size=10000, max_epoch=100000, stopping_criterion='', validation_metrics='', accumulate_gradients=1, num_workers=4, same_nb_ops_per_batch=False, reload_size=-1, env_name='sai', tasks='add_dataset', beam_eval=False, beam_size=1, beam_length_penalty=1, beam_early_stopping=True, reload_model='', reload_checkpoint='', eval_only=False, eval_verbose=0, eval_verbose_print=False, debug_slurm=False, debug=False, cpu=False, local_rank=-1, master_port=-1, my_device='cuda')
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank  

INFO - 01/08/23 23:41:11 - 0:00:00 - Found 370 parameters in model.
INFO - 01/08/23 23:41:12 - 0:00:01 - Optimizers: model


In [2]:
A = 'KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE'
B = 'KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBE'
X = 'KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE'

print(trainer.single_act(A, A))

<s>A44EWWM4444WS444W444W44H44S4DED44SD4AS44A44W


In [5]:
start_time = time.time()

losses = []
iterations_number = 1000

for it in range(iterations_number):
        for _ in range(params.batch_size):
            if random.random() > 0.5:
                flag, loss = trainer.learn(A, B, X, 1)
            else:
                flag, loss = trainer.learn(X, A, B, 1)
        if flag:
            print(it, loss.item())

end_time = time.time()
elapsed_time = end_time - start_time
av_time = elapsed_time/iterations_number

print(f'Av time per iteration per trainer: {elapsed_time/iterations_number:.2f} seconds')


INFO - 01/08/23 23:42:57 - 0:01:46 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=<s>A3BSA4AS444WS44DWSA4<s>44H44SBDAD44SD4AS44AO44, score=0.19148936867713928
INFO - 01/08/23 23:42:57 - 0:01:46 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=<s>A4<s>AWWZA3CD<s>3MAAE3KW<s>44KZC5B3E3L43<s>B<s>2<s>4<s>4EW, score=0.1489361673593521
INFO - 01/08/23 23:42:57 - 0:01:46 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=A43B<s>W<s>AA4ADD34A4B5SW444H44SBDAD44SD4AS44A44D, score=0.1702127605676651
INFO - 01/08/23 23:42:57 - 0:01:46 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=<s>A<s><s>AWWE4I<s>3W3GAAE3A4<s>BOK44KB3E344AZ3<s>3<s>E<s>OE<s>, score=0.1702127605676651
INFO - 01/08/23 23:42:57 - 0:01:46 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=AA<s><s>EW<s><s>3Z<s>3WB4AOZ3A4<s>BJKOC<s>B3E3O453333<s>4<s>AEW, score=0.12765957415103912
INFO - 01/08/23 23:42:57 - 0:01:46 - learning: src=KAAADAAAAADA

0 3.23146915435791


INFO - 01/08/23 23:42:57 - 0:01:46 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=AA3B<s>A<s>AAAA3<s>A3ABB5AS<s>BAK2C<s>B<s>A3<s>3B<s>4A2A4AOA3, score=0.3617021143436432
INFO - 01/08/23 23:42:57 - 0:01:46 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=BA<s>BEB<s>BB3A3<s>3DAABAA2<s>B<s>K3C<s>B<s><s>3<s>3BA3A3E3AABD, score=0.25531914830207825
INFO - 01/08/23 23:42:57 - 0:01:46 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=BA4BAA<s>BAAADBB4AABAAABBAHAABBDADH4AA4BSB4A4BD, score=0.3617021143436432
INFO - 01/08/23 23:42:57 - 0:01:46 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=<s>B<s>BAB<s>EAAA3<s>34B5BAA2<s>BJKBC5B3A3<s>45A3<s>243<s>O<s><s>, score=0.21276596188545227
INFO - 01/08/23 23:42:57 - 0:01:46 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=AAWAAAAAAAABBB4ADBDABBBAHBASBDAD4BSDABSA4A4BD, score=0.3404255211353302
INFO - 01/08/23 23:42:57 - 0:01:46 - learning: src=KAAADAAAAADAAAAA

1 2.8876078128814697


INFO - 01/08/23 23:42:57 - 0:01:47 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=AAABAA<s>ABAABBBAABBAABBBABBA<s>BBA3BBBBBB2B3AAAD, score=0.40425533056259155
INFO - 01/08/23 23:42:57 - 0:01:47 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=AABBBHBBBABBBB4ABBAABBBBBBABBBADBBBBBBSA4ABBD, score=0.21276596188545227
INFO - 01/08/23 23:42:57 - 0:01:47 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=BAABAAAAAAB<s>BAAAABAAA<s>BBKAAABAA3ABABBA2B3AAB3, score=0.5744680762290955
INFO - 01/08/23 23:42:57 - 0:01:47 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=ABBBA<s>BABA33BBAABB5ABBBAKBAABAADBB5ABB2B3AABA, score=0.3404255211353302
INFO - 01/08/23 23:42:57 - 0:01:47 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=BABBAB<s>ABABBBBAABBAAB<s>B<s>BBAKB3A33BBBBB2BBAOB<s>, score=0.27659574151039124
INFO - 01/08/23 23:42:57 - 0:01:47 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=A

...

INFO - 01/08/23 23:44:17 - 0:03:07 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, score=0.8297872543334961
INFO - 01/08/23 23:44:17 - 0:03:07 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, score=0.8297872543334961
INFO - 01/08/23 23:44:17 - 0:03:07 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBE, score=0.08510638028383255
INFO - 01/08/23 23:44:17 - 0:03:07 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, score=0.8297872543334961
INFO - 01/08/23 23:44:17 - 0:03:07 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, score=0.8297872543334961
INFO - 01/08/23 23:44:17 - 0:03:07 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=KAAADAAAAADAAAAAMAAAA

334 0.11912928521633148


INFO - 01/08/23 23:44:17 - 0:03:07 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, score=0.8297872543334961
INFO - 01/08/23 23:44:17 - 0:03:07 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, score=0.8297872543334961
INFO - 01/08/23 23:44:17 - 0:03:07 - learning: src=KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE, pred=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, score=0.8297872543334961
INFO - 01/08/23 23:44:17 - 0:03:07 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBE, score=0.08510638028383255
INFO - 01/08/23 23:44:17 - 0:03:07 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBE, score=0.08510638028383255
INFO - 01/08/23 23:44:17 - 0:03:07 - learning: src=KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE, pred=KBBBBBBBBBBBBBBBBBBB

335 0.11656446754932404


KeyboardInterrupt: 

In [ ]:
torch.cuda.get_device_name(next(trainers[0].modules['transformer'].te1.parameters()).device.index)

In [ ]:
A = 'KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE'
B = 'KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBE'
X = 'KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE'

#rnds = [gen_rnd_chars(xy_data_size_const) for _ in range(100)]

print(trainers[0].act(A))
print(trainers[0].act(B))

In [ ]:
A = 'KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE'
B = 'KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBE'
X = 'KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE'

losses = []
for it in range(2000):
    loss = trainer.learn(join_sai(A), join_sai(A))
    if loss[0]:
        l = loss[1].item()
        losses += [l]

plt.plot(losses)
plt.title('los s vs iteration')
plt.xlabel('iteration')
plt.ylabel('loss')
plt.show()
